# Generating flow duration and flow size

**NOTE** So far with the training, the GAN cannot find any kind of real pattern for the data. It is sometimes produced negative numbers which is not desirable.

Originally, data length has been 1024 in batches of 32. 32^2 = 1024. I plan to lengthen this data to increase the chances of the GAN finding a suitable pattern. 128^2 = 16,384 rows of data.

There will be alternate cells underneath the originals that will correspond with lengthening the training data. These cells will be clearly marked what is the original and what is the alternate. Either version can be turned into a Markdown cell to use the other version, and vice versa. 

In [8]:
import torch 
from torch import nn # nn is neural networks
import pandas as pd
import time

## Discriminator class

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # Input is 2D, first hidden layer is composed of 256 neurons with ReLU activation
            nn.Linear(2, 256), 
            nn.ReLU(),

            # Have to use dropout to avoid overfitting
            nn.Dropout(0.3),

            # second and third layers are composed to 128 and 64 neurons, respectively
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # output is composed of a single neuron with sigmoidal activation to represent a probability
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Generator class 

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Preparing data

### Original

From learning about reproducibility from the PyTorch course, I now understand a bit more how `torch.manual_seed()` works
It adds a certain kind of 'flavour' to the randomness, which allows reproducibility when running the code. 

In [ ]:
# torch.manual_seed(111) Now that I have taken out the random points from the original GAN, this does not need to be here. 
# Instead it should be before every torch.rand() statement

train_data_length = 1024

df = pd.read_csv("archive/dur_size_data.csv")
data = torch.Tensor()

for i in range(len(df)):
    duration = df.loc[i, "Flow.Duration"]
    size = df.loc[i, "Flow.Bytes.s"]
    new_row = torch.Tensor([[duration, size]])
    data = torch.cat([data, new_row])
data.shape

train_labels = torch.zeros(train_data_length)
train_set = [(data[i], train_labels[i]) for i in range(train_data_length)]

# ip_addresses = ip_addresses.detach().numpy()
# test_df_1 = pd.DataFrame(ip_addresses, columns = ["Octet 1", "Octet 2", "Octet 3", "Octet 4"])
# test_df_1.to_csv("Test Data/original_ip_addresses.csv", index = False)

batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
print("done")

### Alternate - Longer training data
* `train_data_length` has now changed to 16384
* Instead of shortening the data before it's read in the file, shorten after it's read
* `batch_size` is now 128

In [11]:
# torch.manual_seed(111)

train_data_length = 16384

df = pd.read_csv("archive/dur_size_data.csv")
df = df[:16384] 
data = torch.Tensor()

for i in range(len(df)):
    duration = df.loc[i, "Flow.Duration"]
    size = df.loc[i, "Flow.Bytes.s"]
    new_row = torch.Tensor([[duration, size]])
    data = torch.cat([data, new_row])
data.shape

train_labels = torch.zeros(train_data_length)
train_set = [(data[i], train_labels[i]) for i in range(train_data_length)]

# ip_addresses = ip_addresses.detach().numpy()
# test_df_1 = pd.DataFrame(ip_addresses, columns = ["Octet 1", "Octet 2", "Octet 3", "Octet 4"])
# test_df_1.to_csv("Test Data/original_ip_addresses.csv", index = False)


batch_size = 128
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
print("done")

done


## Prepare for training

In [12]:
generator = Generator()
discriminator = Discriminator()

# Before training the models, need to define some parameters
learning_rate = 0.001 # used to adapt the network weights
num_epochs = 1000 # how many repetitions of training using the whole training set will be performed
loss_function = nn.BCELoss()
optimiser_discriminator = torch.optim.Adam(discriminator.parameters(), lr = learning_rate)
optimiser_generator = torch.optim.Adam(generator.parameters(), lr = learning_rate)

print(generator.parameters())
print(discriminator.parameters())

<generator object Module.parameters at 0x0000020345125FC0>
<generator object Module.parameters at 0x0000020345125700>


## Training process

In [13]:
start_time = time.time()
from tqdm.notebook import tqdm

for epoch in range(num_epochs):

    # Taking the real samples of the current batch from the data loader and assign them to real_samples
    # The first dimension of the tensor has the number of elements equal to the batch size. 
    # This is the standard way of organising data in PyTorch, with each line of the tensor representing one sample from the batch.
    for n, (real_samples, _) in enumerate(tqdm(train_loader)):
        # DATA FOR TRAINING THE DISCRIMINATOR

        # Using torch.ones() to create labels with the value 1 for real samples, and then assign to real_samples_labels
        real_samples_labels = torch.ones((batch_size, 1))

        # Create the generated samples by storing random data in latent_space_samples
        # This is fed into the generator to obtain generated_samples
        torch.manual_seed(77)
        latent_space_samples = torch.randn((batch_size, 2))
        generated_samples = generator(latent_space_samples)


        # Use torch.zeros() to assign 0 to the labels for the generated samples
        generated_samples_labels = torch.zeros((batch_size, 1))


        # Concatenate the real and generated samples and labels and store them in all_samples
        # and all_samples_labels to train the discriminator
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))



        # TRAINING THE DISCRIMINATOR

        # In PyTorch, it's necessary to clear the gradients at each training step to avoid accumulating them
        discriminator.zero_grad()

        # Calculate the output of the discriminator from the training data in all_samples
        output_discriminator = discriminator(all_samples)

        # Calculate the loss function using discriminator output and all the labels
        loss_discriminator = loss_function(output_discriminator, all_samples_labels)
        
        # Calculate the gradients to update the weights
        loss_discriminator.backward()

        # Update the discriminator weights
        optimiser_discriminator.step()



        # DATA FOR TRAINING THE GENERATOR

        # Storing random data in latent_space_samples with a number of lines to equal batch_size
        torch.manual_seed(77)
        latent_space_samples = torch.randn((batch_size, 2))

        # TRAINING THE GENERATOR
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)

        # Feeding generator's output into the discriminator and store its output, which is used
        # as the output of the whole model
        output_discriminator_generated = discriminator(generated_samples)

        # Calculate the loss function
        loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
        
        # Calculate and update the gradients
        # REMEMBER:
        # When the generator is trained, the discriminator weights are frozen since optimiser_generator
        # was created with its first argument equal to generator.parameters()
        loss_generator.backward()
        optimiser_generator.step()


        # Show loss
        if epoch % 10 == 0 and n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}\n")

end_time = time.time()
run_time = round(end_time - start_time, 2)
print(f"Run time for {num_epochs} epoch(s) is {run_time} seconds.")
print("done")

  0%|          | 0/128 [00:00<?, ?it/s]

Epoch: 0 Loss D.: 17.609865188598633
Epoch: 0 Loss G.: 4.000887870788574



  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

...

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

Run time for 1000 epoch(s) is 966.17 seconds.
done


## Generating synthetic data from the trained adversaries

In [15]:
latent_space_samples = torch.randn(1000, 2)
generated_samples = generator(latent_space_samples)

generated_samples = generated_samples.detach().numpy()
test_df_2 = pd.DataFrame(generated_samples, columns = ["Duration", "Size"])
# test_df_2.to_csv("Test Results/Report Results/Plugged In/generated_dur_size_1000_16384.csv", index = False)
print(generated_samples)

[[ 4.3871452e+01 -1.7235551e+00]
 [ 5.8853540e+02  1.1951101e+01]
 [ 2.7428116e+02 -1.6679544e+00]
 ...
 [ 1.7903477e+02 -5.5889612e-01]
 [ 6.2362671e+02  7.1564789e+00]
 [ 4.2671832e+02  3.1534154e-02]]
